# Logistic regression in pytorch
## What's in this tutorial?
This notebook will walk you through the basic uses of `mandala` for storing
and tracking ML experiment results. It uses logistic regression on a synthetic
dataset as a "minimally interesting" example of a data management use case. By
following this ML mini-project, you will learn how to
- break up an experiment into Python functions whose calls can be
tracked and queried by `mandala`;
- use `mandala`'s memoization to avoid re-running expensive computations and to
naturally interact with and grow your project (by adjusting the parameters and/or
adding new code);
- repurpose the (pure Python) code of your experiments into a *query interface*
to their results "for free";
- change and create new versions of your experimental primitives and have them
  seamlessly interact with the results of previous runs.

Ultimatley, the features of `mandala` work together to enable you to evolve
complex ML projects by writing only the plain-Python code that you'd write in a
temporary in-memory interactive session, yet get the benefits of a
database-backed experiment tracking system. 

## Import libraries

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.utils as utils
import numpy as np

# for reproducibility
np.random.seed(0)
torch.random.set_rng_state(torch.manual_seed(0).get_state())

# recommended way to import mandala functionality
from mandala_lite.all import *

## Define experiment primitives
For the purposes of this tutorial, you'll break the project into two main
functions: to generate the dataset, and to train the model. Below is fairly
standard `pytorch` code for these; note the use of `@op` to mark the functions
as tracked by `mandala`:

In [ ]:
DATA_DIMENSION = 10

# `mandala` decorator; like @functools.lru_cache, but with extra functionality.
# Currently, you must specify the exact number of inputs (i.e., no *args or **kwargs),
# and the number of outputs (via a type annotation).
@op
def generate_dataset() -> Tuple[TensorDataset, TensorDataset]:
    n_samples = 1000
    x = np.random.randn(n_samples, DATA_DIMENSION)
    y = x[:, 0] > 0
    # convert to torch tensors
    x, y = torch.from_numpy(x).float(), torch.from_numpy(y).long()
    # create train/test split with 80/20 ratio
    train_size = int(0.8 * n_samples)
    train_dataset = TensorDataset(x[:train_size], y[:train_size])
    test_dataset = TensorDataset(x[train_size:], y[train_size:])
    return train_dataset, test_dataset

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(DATA_DIMENSION, 2)

    def forward(self, feature):
        output = self.linear(feature)
        return output


@op
def train_model(
    train_dataset: TensorDataset,
    test_dataset: TensorDataset,
    learning_rate: float = 0.001,
    batch_size: int = 100,
    num_epochs: int = 5,
) -> Tuple[LogisticRegression, float]:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # train a logistic regression model with the given loaders and hyperparameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LogisticRegression().to(device)
    loss = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        # train
        model.train()
        for batch_index, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss_value = loss(output, labels)
            loss_value.backward()
        optimizer.step()
        # test
        accurate, total = 0, 0
        model.eval()
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            accurate += (predicted == labels).sum()
        acc = 100 * accurate / total
        print(
            f"Epoch: {epoch}, Training loss: {round(loss_value.item(), 2)}. Test accuracy: {round(acc.item(), 2)}"
        )
    return model, round(float(acc.item()), 2)

## Run the pipeline and log the results
Now that you have defined the functions that make up your pipeline, you can
run it with the default parameters and log the results. 

The `@op` decorator on the functions above tells `mandala` to track the calls to
these functions, and to store the results - but this only happens when you call
these functions *in the context of a given `Storage` object*. So go ahead, and
create a storage for the project: 

In [ ]:
storage = Storage()

This storage will hold the results of all the experiments you run in this
notebook. Now, run the pipeline by wrapping the code you'd normally write in a 
`storage.run()` context manager; `mandala` will automatically track the calls to
the functions you marked with `@op` and store the results in the `storage`
object:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    model, acc = train_model(train_dataset, test_dataset)
    print(f"Final accuracy: {acc}")

### What just happened?
A lot happened behind the scenes in these few lines of code! Let's break it
down:
- Inside the `storage.run()` block, each time an `@op`-decorated function is
called **for the first time** on a set of inputs, `mandala` stores the inputs
and outputs of this call in the storage. 
- Values shared between calls are stored only once. So
  `train_dataset` will appear in storage as both the output to the call to
  `generate_dataset`, and the input to the call to `train_model`.
- The `acc` object (like all objects returned by `@op`-decorated functions) is a
*value reference*, which is a value wrapped with storage-related metadata. 

So, what happens when you call `@op`-decorated functions *a second time* on the
same inputs? Find out by running the cell below:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    model, acc = train_model(train_dataset, test_dataset)
    print(f"Final accuracy: {acc}")

**Note that this time the intermediate training results did not get printed!**.
This is because `mandala` recognized that the inputs to the functions were the
same as before, and so it didn't need to re-run the calls. This is also evident
from the lack of output from the model training!

## Grow the project with new parameters
Running the pipeline once is nice, but where `mandala` really shines is in
enabling you to grow a computational project in various ways with the minimal
necessary code changes, and have the storage interfaces "just work". 

Let's begin exploring this by investigating the effect of changing the learning
rate of the model. So far, you have been using the default learning rate of
`0.001`. Let's try a few other values, but also see how they compare with the
default value. Thanks to memoization, this is easy to do without re-doing
expensive work: we can use a list of values for the `learning_rate` parameter
that includes the default, and compare:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for learning_rate in [0.001, 0.01, 0.1]:
        model, acc = train_model(train_dataset, test_dataset, learning_rate)
        print(
            f"===end of run=== learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
        )

Note that the first run was re-used from before, while the 2nd and 3rd were
freshly computed. We see that the higher the learning rate, the better the final
accuracy.  Now, let's try varying the batch size as well:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.001, 0.01, 0.1]:
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

## Query the results
By now, you have run the pipeline with many different combinations of
parameters, and it's getting difficult to make sense of all the results so far.
One option to get to the results is to just re-run the above workflow, or a
"sub-workflow" of it. 

For example, how might you get all the results for a given learning rate, e.g.
`learning_rate=0.1`? One answer: just by re-running the subset of the above code
using this value of the learning rate:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.1]:  # only change relative to previous cell
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

This kind of storage access pattern is called **retracing**: you "retrace"
computational code that you have already run in order to recover the quantities
computed along the way. You can use retracing to query existing results (like
you did above), or to easily add new parameters/logic that need to compute over
existing results.

However, sometimes you don't have a specific piece of code to retrace and just
want to look at all the results in storage. For this, you can use a "full
storage search" query interface via the `storage.query()` context manager:

In [ ]:
with storage.query() as q:
    train_dataset, test_dataset = generate_dataset()
    batch_size = Q().named("batch_size")
    learning_rate = Q().named("learning_rate")
    model, acc = train_model(train_dataset, test_dataset, learning_rate, batch_size)
    df = q.get_table(batch_size, learning_rate, acc.named("accuracy"))

df.sort_values(by=["accuracy"], ascending=False)

### What just happened?
In the `storage.query()` context manager, calls to `@op`-decorated functions
build a computational graph of variables and constraints behind the scenes:
- **variables** stand for a value stored in storage. These include the `Q()`
objects above, as well as the values returned by `@op`-decorated functions.
- **constraints** (i.e. function calls) enforce relationships between variables.

The result of the query is a table, where each row is a choice of values for 
the variables that satisfy the constraints.

## Try a larger dataset, or: how to modify memoized functions gracefully
A very common scenario in ML is to extend an existing experimental primitive
with new functionality, for example by exposing a hard-coded parameter, or
adding a new behavior to the function (e.g., an option to use a different
algorithm). `mandala` allows you to do this gracefully, seamlessly integrating 
the new functionality with the results of previous runs.

For example, suppose you want to improve the accuracy numbers above. So far, you
have been running all models on a synthetic dataset of size `1000`. Let's try
increasing the dataset size while still sampling from the same distribution and
see if that helps! 

To do this, simply modify the `generate_dataset` function to take an additional
argument `n_samples` with a default value of `1000`. Let's also print out a
message when it's called:

In [ ]:
@op
def generate_dataset(n_samples: int = 1000) -> Tuple[TensorDataset, TensorDataset]:
    print("Hi from `generate_dataset` with an extra argument!")
    x = np.random.randn(n_samples, DATA_DIMENSION)
    y = x[:, 0] > 0
    # convert to torch tensors
    x, y = torch.from_numpy(x).float(), torch.from_numpy(y).long()
    # create train/test split with 80/20 ratio
    train_size = int(0.8 * n_samples)
    train_dataset = TensorDataset(x[:train_size], y[:train_size])
    test_dataset = TensorDataset(x[train_size:], y[train_size:])
    return train_dataset, test_dataset

What happens to the memoized calls after we change the function? Find out by
re-running the project so far:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset()
    for batch_size in [100, 200, 400]:
        for learning_rate in [0.001, 0.01, 0.1]:
            model, acc = train_model(
                train_dataset, test_dataset, learning_rate, batch_size
            )
            print(
                f"===end of run=== batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
            )

As you can see, `generate_dataset` was quiet, indicating that it was not
re-run! This is because `mandala` recognized that the inputs to the function
were the default values, and so it didn't need to re-run the call. Note that you
must make the default values consistent with the previous runs of the function;
alternatively, you can set them to `None` and deal with this logic inside the
function. 

Now let's finally run the pipeline with a larger dataset:

In [ ]:
with storage.run():
    for n_samples in [1000, 2000]:
        train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
        for batch_size in [100, 200, 400]:
            for learning_rate in [0.001, 0.01, 0.1]:
                model, acc = train_model(
                    train_dataset, test_dataset, learning_rate, batch_size
                )
                print(
                    f"===end of run=== n_samples: {n_samples}, batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
                )

Let's also get a nice table with the results; you can do this by minimally
modifying the query code from before!

In [ ]:
with storage.query() as q:
    n_samples = Q().named("n_samples")
    train_dataset, test_dataset = generate_dataset(n_samples)
    batch_size = Q().named("batch_size")
    learning_rate = Q().named("learning_rate")
    model, acc = train_model(train_dataset, test_dataset, learning_rate, batch_size)
    df = q.get_table(n_samples, batch_size, learning_rate, acc.named("accuracy"))

df.sort_values(by=["accuracy"], ascending=False)

Hmmm, it looks like the larger dataset didn't help. Why might that be?

## "Oh no, there's a bug!", or: function versioning
If you look at the `train_model` function carefully, you'll see there's a
subtle but crucial mistake: because the call to `optimizer.step()` happens
outside the loop over the batches, the model is only updated once per epoch!

Unfortunately, we have already ran a lot of experiments with this bug. How can
we fix this without messing up our storage and re-running more than we need to?
Starting the whole project from scratch is not great, because it means we would
have to regenerate the dataset too. 

Fortunately, `mandala` has a simple way to deal with this situation with the
minimal possible amount of code changes. You must do two things:
- change the function to fix the bug, and **increment the
  version number**. By default, each function starts at version `0`. 
- re-run the experiments that used the old version of the function. All results
  that do not depend on the bug will be reused, and only the results that
  depended on the bug will be recomputed (if the bug fix happens to lead to
  different values of the outputs).

So, fix the bug and increment the version number in the `@op` decorator:

In [ ]:
@op(version=1)
def train_model(
    train_dataset: TensorDataset,
    test_dataset: TensorDataset,
    learning_rate: float = 0.001,
    batch_size: int = 100,
    num_epochs: int = 5,
) -> Tuple[LogisticRegression, float]:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # train a logistic regression model with the given loaders and hyperparameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LogisticRegression().to(device)
    loss = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        # train
        for batch_index, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss_value = loss(output, labels)
            loss_value.backward()
            optimizer.step()
        # test
        accurate, total = 0, 0
        for images, labels in test_loader:  #! bugfix here
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            accurate += (predicted == labels).sum()
        acc = 100 * accurate / total
        print(
            f"Epoch: {epoch}, Training loss: {round(loss_value.item(), 2)}. Test accuracy: {round(acc.item(), 2)}"
        )
    return model, round(float(acc.item()), 2)

Now, literally copy-paste the computation + query code from above!

In [ ]:
with storage.run():
    for n_samples in [1000, 2000]:
        train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
        for batch_size in [100, 200, 400]:
            for learning_rate in [0.001, 0.01, 0.1]:
                model, acc = train_model(
                    train_dataset, test_dataset, learning_rate, batch_size
                )
                print(
                    f"===end of run=== n_samples: {n_samples}, batch_size: {batch_size}, learning_rate: {learning_rate}, acc: {round(unwrap(acc), 2)}"
                )

with storage.query() as q:
    n_samples = Q().named("n_samples")
    train_dataset, test_dataset = generate_dataset(n_samples)
    batch_size = Q().named("batch_size")
    learning_rate = Q().named("learning_rate")
    model, acc = train_model(train_dataset, test_dataset, learning_rate, batch_size)
    df = q.get_table(n_samples, batch_size, learning_rate, acc.named("accuracy"))

df.sort_values(by=["accuracy"], ascending=False)

OK! Now we have the correct results. Maybe. At least, they make more sense than
before: accuracy went up, and the larger dataset helped.

## Compare model mistakes, or: easily track workflows with complex logic
So far, you have been running the same two-part pipeline independently for
several sets of parameters. This kind of workflow is common in ML projects, and
many libraries provide e.g. hyperparameter tuning tools for such cases (e.g., 
`sklearn`). 

However, real-world experiments often go beyond these "embarrassingly DAG-like"
workflows, and require more complex logic. This is another place where `mandala`
shines: thanks to the transparent way you build experiments directly in plain
Python, you have full control over the logic of your experiments, while getting
the benefits of a storage system that "just works". 

To illustrate this, let's compare the mistakes made by models trained with the
same batch size and dataset, but different learning rates. First, define a
function that computes the Jaccard index between the sets of mistakes of two
models:

In [ ]:
@op
def compare_mistakes(
    model_1: LogisticRegression,
    model_2: LogisticRegression,
    test_dataset: TensorDataset,
) -> float:
    # compute the Jaccard index between the mistakes of the two models
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_1 = model_1.to(device)
    model_2 = model_2.to(device)
    model_1.eval()
    model_2.eval()
    mistakes_1 = set()
    mistakes_2 = set()
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output_1 = model_1(images)
        output_2 = model_2(images)
        _, predicted_1 = torch.max(output_1.data, 1)
        _, predicted_2 = torch.max(output_2.data, 1)
        for i in range(len(labels)):
            if predicted_1[i] != labels[i]:
                mistakes_1.add(i)
            if predicted_2[i] != labels[i]:
                mistakes_2.add(i)
    return round(
        len(mistakes_1.intersection(mistakes_2)) / len(mistakes_1.union(mistakes_2)), 2
    )

Now, compare the mistakes of all the models trained on the synthetic dataset of
size `2000` for each batch size:

In [ ]:
with storage.run():
    train_dataset, test_dataset = generate_dataset(n_samples=2_000)
    for batch_size in [100, 200, 400]:
        for lr_1 in [0.001, 0.01, 0.1]:
            for lr_2 in [0.001, 0.01, 0.1]:
                if lr_1 <= lr_2:
                    continue
                model_1, acc_1 = train_model(
                    train_dataset, test_dataset, lr_1, batch_size
                )
                model_2, acc_2 = train_model(
                    train_dataset, test_dataset, lr_2, batch_size
                )
                jaccard_index = compare_mistakes(model_1, model_2, test_dataset)
                print(
                    f"===end of run=== batch_size: {batch_size}, lr_1: {lr_1}, lr_2: {lr_2}, jaccard_index: {unwrap(jaccard_index)}"
                )

Importantly, note that thanks to the memoization, the only new computations here
were calls to the new `compare_mistakes` function! Despite rearranging the code
a bit instead of just copy-pasting it, the results are still reused.

### Query the new workflow
How might you get a nice table with the results of the above workflow? Again,
you have two options. 

The simplest is just to retrace the code that you have already run, and collect 
the results as rows of a dataframe:

In [ ]:
with storage.run():
    rows = []
    train_dataset, test_dataset = generate_dataset(n_samples=2_000)
    for batch_size in [100, 200, 400]:
        for lr_1 in [0.001, 0.01, 0.1]:
            for lr_2 in [0.001, 0.01, 0.1]:
                if lr_1 <= lr_2:
                    continue
                model_1, acc_1 = train_model(
                    train_dataset, test_dataset, lr_1, batch_size
                )
                model_2, acc_2 = train_model(
                    train_dataset, test_dataset, lr_2, batch_size
                )
                jaccard = compare_mistakes(model_1, model_2, test_dataset)
                rows.append(
                    {
                        "batch_size": batch_size,
                        "lr_1": lr_1,
                        "lr_2": lr_2,
                        "jaccard": unwrap(jaccard),
                    }
                )
df = pd.DataFrame(rows)
df.sort_values(by=["jaccard"], ascending=False)

You can also use the `storage.query()` context manager, but you need to be
careful to specify the correct constraints! In particular, you must create two
placeholders for the learning rate (why?). 

The easiest way to build a query is to copy-paste the computational code, and
then modify it to replace iteration over a parameter range with a placeholder:

In [ ]:
with storage.query() as q:
    n_samples = Q().named("n_samples")
    train_dataset, test_dataset = generate_dataset(n_samples=n_samples)
    batch_size = Q().named("batch_size")
    lr_1 = Q().named("lr_1")
    lr_2 = Q().named("lr_2")
    model_1, acc_1 = train_model(train_dataset, test_dataset, lr_1, batch_size)
    model_2, acc_2 = train_model(train_dataset, test_dataset, lr_2, batch_size)
    jaccard_index = compare_mistakes(model_1, model_2, test_dataset)
    df = q.get_table(n_samples, batch_size, lr_1, lr_2, jaccard_index.named("jaccard"))

df.sort_values(by=["jaccard"], ascending=False)

## Conclusion
### The importance of a good function decomposition
Decomposing a project like this one into functions (i.e., `generate_dataset` and
`train_model`) is something you often find yourself doing almost without
thinking. It's a necessary practice in software engineering and ML
experimentation in general. 

However, it is also particularly important to find a "good" decomposition when
tracking experiments with `mandala`, because the kind of decomposition you use
has consequences on the performance of your storage system.

For example, consider the alternative of having a single function that does both
the dataset generation and the model training:
```python
def generate_and_train(n_samples, batch_size, learning_rate) -> Tuple[LogisticRegression, float]:
    ...
```

A frequent use case is to train many models on the same dataset, but with
different hyperparameters. With a single function, you'd have to re-run the
dataset generation function every time, even though the dataset is the same!